In [1]:
import pandas as pd

OUTPUT_DATABASE = "../data/profile_layers.csv"
EVA_LARGE_PATCH14_448_MIM = "eva02_large_patch14_448.mim_in22k_ft_in22k_in1k"
VIT_HUGE_PATCH14_CLIP_224 = "vit_huge_patch14_clip_224.laion2b_ft_in12k_in1k"


# Transformers models characteristics

In [2]:
df_trans = pd.read_csv(OUTPUT_DATABASE)

In [3]:
df_trans

,net,layer,var_name,layer_params,depth,output_size
0,resnet50d,ResNet,ResNet,25576264,0,1000
1,resnet50d,Sequential,conv1,28640,1,802816
2,resnet50d,Conv2d,0,864,2,401408
3,resnet50d,BatchNorm2d,1,64,2,401408
4,resnet50d,ReLU,2,0,2,401408
...,...,...,...,...,...,...
10790,eva02_base_patch14_448.mim_in22k_ft_in22k_in1k,Dropout,drop2,0,4,787200
10791,eva02_base_patch14_448.mim_in22k_ft_in22k_in1k,Identity,drop_path2,0,3,787200
10792,eva02_base_patch14_448.mim_in22k_ft_in22k_in1k,Identity,norm,0,1,787200
10793,eva02_base_patch14_448.mim_in22k_ft_in22k_in1k,LayerNorm,fc_norm,1536,1,768


In [4]:
df_trans["var_name_layer"] =  df_trans["var_name"]
df_trans.loc[df_trans["layer"].str.lower().str.contains("block"), "var_name_layer"] = "Block"


In [5]:
var_names = ['norm', 'attn', 'block', 'mlp', 'stage', "swiglu", "gelu", "act"]
# filter the dataframe by var_name substrings
df_filtered = df_trans[df_trans['var_name_layer'].str.lower().str.contains('|'.join(var_names))]

# group by 'net' and 'var_name', and get the index of the row with the highest 'layer_params'
idx = df_filtered.groupby(['net', 'var_name_layer'])['output_size'].idxmax()

# select the rows with the highest 'layer_params' using the index
result = df_filtered.loc[idx]

In [6]:
# As we cannot use all of them select two. Best accuracy
# -- vit_huge_patch14_clip_224.laion2b_ft_in12k_in1k 88.3%
# -- eva02_large_patch14_448.mim_in22k_ft_in22k_in1k 89.94
subset_of_models = [EVA_LARGE_PATCH14_448_MIM, VIT_HUGE_PATCH14_CLIP_224]

In [7]:
result = result[
    (result["layer"] != "Identity") &
    (result["layer"] != "Dropout") &
    (result["layer"] != "Sequential") &

    (result["net"].isin(subset_of_models))
    ]

In [8]:
result = result[~result["var_name"].isin(["norm1", "norm2", "norm_pre"])]

In [9]:
df_trans

,net,layer,var_name,layer_params,depth,output_size,var_name_layer
0,resnet50d,ResNet,ResNet,25576264,0,1000,ResNet
1,resnet50d,Sequential,conv1,28640,1,802816,conv1
2,resnet50d,Conv2d,0,864,2,401408,0
3,resnet50d,BatchNorm2d,1,64,2,401408,1
4,resnet50d,ReLU,2,0,2,401408,2
...,...,...,...,...,...,...,...
10790,eva02_base_patch14_448.mim_in22k_ft_in22k_in1k,Dropout,drop2,0,4,787200,drop2
10791,eva02_base_patch14_448.mim_in22k_ft_in22k_in1k,Identity,drop_path2,0,3,787200,drop_path2
10792,eva02_base_patch14_448.mim_in22k_ft_in22k_in1k,Identity,norm,0,1,787200,norm
10793,eva02_base_patch14_448.mim_in22k_ft_in22k_in1k,LayerNorm,fc_norm,1536,1,768,fc_norm
